In [1]:
# Import the driver Overlay class
from axi_stream_driver import NeuralNetworkOverlay

In [5]:
import numpy as np
X_test = np.load('X_test_NRN.npy')
y_test = np.load('y_test_NRN.npy')
X_test = X_test[:50]*256
y_test = y_test[:50]

In [23]:
nn = NeuralNetworkOverlay('ILA_2.bit', X_test[0].shape, y_test[0].shape)

In [27]:
print(X_test[0])

[[[165. 213. 240.]
  [170. 218. 242.]
  [167. 216. 238.]
  ...
  [ 40.  36.  36.]
  [ 50.  49.  52.]
  [ 57.  57.  57.]]

 [[167. 214. 239.]
  [171. 219. 240.]
  [168. 217. 237.]
  ...
  [ 35.  31.  31.]
  [ 60.  59.  62.]
  [ 68.  66.  66.]]

 [[168. 217. 239.]
  [168. 216. 235.]
  [168. 217. 234.]
  ...
  [ 48.  46.  47.]
  [ 72.  69.  72.]
  [ 65.  60.  62.]]

 ...

 [[152. 154. 152.]
  [154. 154. 150.]
  [158. 156. 151.]
  ...
  [141. 147. 141.]
  [136. 144. 143.]
  [137. 147. 149.]]

 [[157. 156. 152.]
  [159. 157. 151.]
  [158. 157. 153.]
  ...
  [138. 144. 140.]
  [138. 144. 144.]
  [140. 146. 148.]]

 [[161. 157. 151.]
  [163. 159. 151.]
  [158. 159. 156.]
  ...
  [141. 147. 144.]
  [143. 147. 148.]
  [145. 146. 148.]]]


In [35]:
y_hw, latency, throughput = nn.predict(X_test[0], debug=True, profile=True)

Input size=3072
Input Buffer: 
 [[[165. 213. 240.]
  [170. 218. 242.]
  [167. 216. 238.]
  ...
  [ 40.  36.  36.]
  [ 50.  49.  52.]
  [ 57.  57.  57.]]

 [[167. 214. 239.]
  [171. 219. 240.]
  [168. 217. 237.]
  ...
  [ 35.  31.  31.]
  [ 60.  59.  62.]
  [ 68.  66.  66.]]

 [[168. 217. 239.]
  [168. 216. 235.]
  [168. 217. 234.]
  ...
  [ 48.  46.  47.]
  [ 72.  69.  72.]
  [ 65.  60.  62.]]

 ...

 [[152. 154. 152.]
  [154. 154. 150.]
  [158. 156. 151.]
  ...
  [141. 147. 141.]
  [136. 144. 143.]
  [137. 147. 149.]]

 [[157. 156. 152.]
  [159. 157. 151.]
  [158. 157. 153.]
  ...
  [138. 144. 140.]
  [138. 144. 144.]
  [140. 146. 148.]]

 [[161. 157. 151.]
  [163. 159. 151.]
  [158. 159. 156.]
  ...
  [141. 147. 144.]
  [143. 147. 148.]
  [145. 146. 148.]]]

Transfer OK
Send OK
Receive OK
Ouput Buffer: 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Classified 32 samples in 0.060434999999999996 seconds (529.4944982212295 inferences / s)


In [33]:
from pynq.lib.video import *
#from pynq.lib.video import hierarchies

In [ ]:
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
#hdmi_out = base.video.hdmi_out
hdmi_out = nn.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

In [ ]:
# initialize camera from OpenCV
import cv2 as cv

videoIn = cv.VideoCapture(-1)
videoIn.set(cv.CAP_PROP_FRAME_WIDTH, 640);
videoIn.set(cv.CAP_PROP_FRAME_HEIGHT, 480);

print("Capture device is open: " + str(videoIn.isOpened()))

In [ ]:
# Capture webcam video and display to HDMI Output
#import numpy as np
x,y,w,h = 300,0,100,40
import time
start = time.time()
for NumOfFrames in range (100): 
    ret, frame = videoIn.read()
    if (ret):
        outframe = hdmi_out.newframe()
        cropped_frame = frame[0:480, 0:480]
        resized = cv.resize(cropped_frame, (32, 32))
        y_hw, latency, throughput = nn.predict(resized, debug=True, profile=True)
        outframe[0:480,0:640,:] = frame[0:480,0:640,:]        
        cv.rectangle(outframe, (x,y), (x + w, y + h), (0,0,0), -1)
        cv.putText(img=outframe, text="CAT",org=(x+int(w/10),y+int(4*h/5)), fontFace=cv.FONT_HERSHEY_DUPLEX, fontScale=1, color=(255,0,0), thickness=1)
        #outframe[0:32,0:32,:] = crop_frame[0:32,0:32,:]
        #outframe = cv.resize(crop_frame,None,fx=(1/15),fy=(1/15),interpolation=cv.INTER_AREA)
        hdmi_out.writeframe(outframe)
    else:
        raise RuntimeError("Failed to read from camera.")
end = time.time()
print("Frames per second:     " + str(50 / (end - start)))